# Imports

In [2]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime

In [4]:
from datetime import date

In [5]:
# 34234
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 as pg
from datetime import datetime
import json
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import datetime
pio.renderers.default = 'notebook'
import warnings
import numpy_financial as npf

warnings.filterwarnings("ignore")
from datetime import timedelta
import numpy_financial as npf

ModuleNotFoundError: No module named 'psycopg2'

In [6]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json
from tqdm import tqdm
import numpy as np
import warnings
from datetime import datetime, timedelta
import ast

import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

/var/folders/br/rqk8kpt50q1_dkr0c2p_53lh0000gp/T/ipykernel_70014/1447463556.py:13: DeprecationWarning: This package has been renamed to snowflake_uuid and will be removed shortly. Please update immediately.
  import snowflake.connector


ModuleNotFoundError: No module named 'snowflake.connector'; 'snowflake' is not a package

In [6]:
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from hampel import hampel
from queries import *

In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [6]:
def query_for_data(query):
    engine = create_engine(URL(user=my_user,password=my_pass,account=my_account,warehouse=my_warehouse,database=my_db))
    connection = engine.connect()
    df = pd.read_sql(query, connection)
    return df

In [7]:
# Function to categorize 'ln' values
def categorize_ln(ln):
    if ln == 0:
        return 'ln0'
    elif ln == 1:
        return 'ln1'
    elif 2 <= ln <= 4:
        return 'ln2-4'
    elif 5 <= ln <= 8:
        return 'ln5-8'
    elif 9 <= ln <= 12:
        return 'ln9-12'
    elif 13 <= ln <= 20:
        return 'ln13-20'
    else:  # ln > 20
        return 'ln21+'


# Load Models

In [8]:
ln_groups = ['ln0', 'ln1', 'ln13_20', 'ln2_4', 'ln21', 'ln5_8', 'ln9_12','portfolio']

In [9]:
# !pip install scikit-learn==0.24.2



In [105]:
model_dict_fdr30_dpd1 = {}
for ln_group in ln_groups:
    file_name = f'fdr30_forecast_for_dpd1_{ln_group}.pkl'
    
    with open(file_name, 'rb') as file:
        model_dict_fdr30_dpd1[f'fdr30_forecast_for_dpd1_{ln_group}_model'] = pickle.load(file)

In [107]:
import pickle
from sklearn.ensemble import GradientBoostingRegressor

# Fix loaded models
for ln_group in ln_groups:
    model_key = f'fdr30_forecast_for_dpd1_{ln_group}_model'
    model = model_dict_fdr30_dpd1[model_key]

    # Check and fix missing attribute
    if isinstance(model, GradientBoostingRegressor):
        if not hasattr(model, 'n_features_'):  # Add n_features_ if missing
            model.n_features_ = model.n_features_in_

        # Optional: Also remove deprecated attributes if still present
        if hasattr(model, 'min_impurity_split'):
            delattr(model, 'min_impurity_split')

        print(f"Fixed model: {model_key}")


Fixed model: fdr30_forecast_for_dpd1_ln0_model
Fixed model: fdr30_forecast_for_dpd1_ln1_model
Fixed model: fdr30_forecast_for_dpd1_ln2_4_model
Fixed model: fdr30_forecast_for_dpd1_ln5_8_model
Fixed model: fdr30_forecast_for_dpd1_ln9_12_model
Fixed model: fdr30_forecast_for_dpd1_ln13_20_model
Fixed model: fdr30_forecast_for_dpd1_ln21_model
Fixed model: fdr30_forecast_for_dpd1_portfolio_model


# Load SF Data

In [42]:
my_user="AMIT_S"
my_pass="Amit2525!!"
my_account="xukvgcv-xm56910"
my_warehouse='COMPUTE_WH'
my_db='ghana_prod'


In [44]:
# FDR1
query_fdr1 = '''
    select ln, loan_id, disbursementdate, REPAYMENT_DUE_DATE,installment, max(disbursed_amount) as disbursed_amount, least(sum(amount), max(disbursed_amount)) repaid_amount1
    from   (
        select 
        ln,
       loan_id, disbursementdate, REPAYMENT_DUE_DATE, disbursed_amount,
        total_disbursed_amount, installment, TRANSACTION_DATE,
case when datediff('day', REPAYMENT_DUE_DATE, TRANSACTION_DATE) <= 1 then amount else 0 end as amount
            from
                (
                    select re.loan_id,
                       truncate(re.ln) ln,
                       re.installment,
                       adjusted_PRINCIPAL_DUE AS disbursed_amount,
                       re.REPAYMENT_DUE_DATE,
                       re.LOANAMOUNT AS total_disbursed_amount,
                       re.disbursementdate,
                       nvl(re.amount, 0) as amount,
                       re.TRANSACTION_DATE
            from ML.LOAN_INFO_TBL ml 
            join     GHANA_PROD.ML.REPAYMENT_TRANSACTIONS_EXTENDED re
              on ml.loan_id=re.loan_id             
            where 
             ml.loan_product_id not like '%MAL%' 
            and re.disbursementdate::date >= '2024-04-01'
              ) t
            where datediff('day', REPAYMENT_DUE_DATE,current_date) > 1 
   order by TRANSACTION_DATE, installment
            ) t2
group by 1, 2, 3, 4,5

'''
query_results_fdr1 = query_for_data(query_fdr1)

In [45]:
# round((1 - sum(repaid_amount)/sum(disbursed_amount))*100,2) as fin_default,
query_results_fdr1['fdr1']= 1 - query_results_fdr1.repaid_amount1/query_results_fdr1.disbursed_amount
query_results_fdr1.head(3)

,ln,loan_id,disbursementdate,repayment_due_date,installment,disbursed_amount,repaid_amount1,fdr1
0,3,GUKP230,2024-11-11 15:23:49,2024-12-11,1,540.0,540.0,0.0
1,5,BOQR407,2024-11-15 09:18:39,2024-12-13,1,850.0,850.0,0.0
2,21,WKMJ771,2024-11-09 13:07:55,2024-12-12,1,3000.0,3000.0,0.0


In [46]:
query_results_fdr1.shape

(1393547, 8)

In [47]:
merged=query_results_fdr1

In [48]:
# Applying the categorize_ln function to create 'ln_group' column
merged['ln_group'] = merged['ln'].apply(categorize_ln)
merged['disbursementdate'] = pd.to_datetime(merged['disbursementdate'])
merged['disbursed_month'] = merged['disbursementdate'].dt.to_period('M')

### Add Due Period

In [50]:
# Function to categorize due dates
def categorize_due_date(date):
    day = date.day
    if day <= 5:
        return 1
    elif 6 <= day <= 25:
        return 2
    else:
        return 3

In [51]:
# Convert 'repayment_due_date' to datetime
merged['repayment_due_date'] = pd.to_datetime(merged['repayment_due_date'])
merged['due_period_manual'] = merged['repayment_due_date'].apply(categorize_due_date)


In [52]:
grouped = merged.groupby(['disbursed_month', 'ln_group', 'due_period_manual']).size().unstack(fill_value=0)

# Calculate total counts for each group
grouped['total'] = grouped.sum(axis=1)

# Calculate proportions for each due_period_manual
grouped['bom_proportion'] = grouped[1] / grouped['total'] * 100
grouped['middle_month_proportion'] = grouped[2] / grouped['total'] * 100
grouped['eom_proportion'] = grouped[3] / grouped['total'] * 100

grouped.head(2)

due_period_manual            1     2     3  total  bom_proportion  \
disbursed_month ln_group                                            
2024-04         ln0       1120  8502  2580  12202        9.178823   
                ln1       1091  5607  1723   8421       12.955706   

due_period_manual         middle_month_proportion  eom_proportion  
disbursed_month ln_group                                           
2024-04         ln0                     69.677102       21.144075  
                ln1                     66.583541       20.460753

In [53]:
grouped2 = merged.groupby(['disbursed_month', 'due_period_manual']).size().unstack(fill_value=0)

# Calculate total counts for each group
grouped2['total'] = grouped2.sum(axis=1)

# Calculate proportions for each due_period_manual
grouped2['bom_proportion'] = grouped2[1] / grouped2['total'] * 100
grouped2['middle_month_proportion'] = grouped2[2] / grouped2['total'] * 100
grouped2['eom_proportion'] = grouped2[3] / grouped2['total'] * 100

In [54]:
grouped_rel=grouped.reset_index()[['disbursed_month','ln_group','bom_proportion','middle_month_proportion','eom_proportion']]
grouped2_rel=grouped2[['bom_proportion','middle_month_proportion','eom_proportion']]

# Apply Models

### Predict FDR30 For Next Month

In [57]:
# data for fdr90
data_test=merged[merged.repayment_due_date<='2025-01-01'] #those that reached fdr1
data_test.shape

(1377575, 11)

In [58]:
data_test.disbursementdate.min()

Timestamp('2024-04-01 00:00:22')

In [59]:
data_test.fdr1.isna().sum()

120

In [60]:
data_test.disbursed_month.value_counts()

disbursed_month
2024-07    186779
2024-05    177246
2024-08    175063
2024-06    164607
2024-04    164552
2024-10    162319
2024-09    160055
2024-11    135351
2024-12     51603
Freq: M, Name: count, dtype: int64

In [61]:
# take only months with min 100,000 loans
data_test=data_test[data_test.disbursed_month=='2024-10']
data_test.shape

(162319, 11)

In [62]:
data_test=data_test.reset_index()

In [79]:
# Create portfolio
# LN Group FDRS
portfolio_df_lns=pd.DataFrame(data_test.groupby(['disbursed_month','ln_group']).disbursed_amount.sum())
portfolio_df_lns['repaid_amount1']=data_test.groupby(['disbursed_month','ln_group']).repaid_amount1.sum()
portfolio_df_lns=round(portfolio_df_lns)

# Calculate the FDR for each time period
portfolio_df_lns['fdr1'] = (1 - portfolio_df_lns['repaid_amount1'] / portfolio_df_lns['disbursed_amount'])*100

portfolio_df_lns=round(portfolio_df_lns,3).reset_index()
portfolio_df_lns=pd.merge(portfolio_df_lns,grouped_rel,how='left',left_on=['disbursed_month','ln_group'],right_on=['disbursed_month','ln_group'])
portfolio_df_lns

,disbursed_month,ln_group,disbursed_amount,repaid_amount1,fdr1,bom_proportion,middle_month_proportion,eom_proportion
0,2024-10,ln0,4833977.0,3389862.0,29.874,18.325386,65.692458,15.982157
1,2024-10,ln1,4566943.0,3610004.0,20.954,17.375246,62.742633,19.882122
2,2024-10,ln13-20,50883680.0,46123912.0,9.354,47.318425,12.619566,40.062009
3,2024-10,ln2-4,13919668.0,11827125.0,15.033,20.204989,58.212410,21.582602
4,2024-10,ln21+,71743360.0,67776442.0,5.529,44.092488,12.972730,42.934783
5,2024-10,ln5-8,20289931.0,18130850.0,10.641,42.837838,17.994088,39.168074
6,2024-10,ln9-12,25628793.0,23194776.0,9.497,46.620116,12.430050,40.949834


In [81]:
# Portfolio FDRS
portfolio_df=pd.DataFrame(data_test.groupby('disbursed_month').disbursed_amount.sum())
portfolio_df['repaid_amount1']=data_test.groupby('disbursed_month').repaid_amount1.sum()
portfolio_df=round(portfolio_df)

# Calculate the FDR for each time period
portfolio_df['fdr1'] = (1 - portfolio_df['repaid_amount1'] / portfolio_df['disbursed_amount'])*100
portfolio_df['ln_group']='Portfolio'
portfolio_df=pd.merge(portfolio_df,grouped2_rel,how='left',left_on=['disbursed_month'],right_index=True)
round(portfolio_df,3)

,disbursed_amount,repaid_amount1,fdr1,ln_group,bom_proportion,middle_month_proportion,eom_proportion
disbursed_month,,,,,,,
2024-10,191866352.0,174052970.0,9.284,Portfolio,35.754,31.124,33.121


In [83]:
portfolio_df_lns=pd.concat([portfolio_df_lns,portfolio_df.reset_index()])
portfolio_df_lns

,disbursed_month,ln_group,disbursed_amount,repaid_amount1,fdr1,bom_proportion,middle_month_proportion,eom_proportion
0,2024-10,ln0,4833977.0,3389862.0,29.874000,18.325386,65.692458,15.982157
1,2024-10,ln1,4566943.0,3610004.0,20.954000,17.375246,62.742633,19.882122
2,2024-10,ln13-20,50883680.0,46123912.0,9.354000,47.318425,12.619566,40.062009
3,2024-10,ln2-4,13919668.0,11827125.0,15.033000,20.204989,58.212410,21.582602
4,2024-10,ln21+,71743360.0,67776442.0,5.529000,44.092488,12.972730,42.934783
5,2024-10,ln5-8,20289931.0,18130850.0,10.641000,42.837838,17.994088,39.168074
6,2024-10,ln9-12,25628793.0,23194776.0,9.497000,46.620116,12.430050,40.949834
0,2024-10,Portfolio,191866352.0,174052970.0,9.284266,35.754176,31.124344,33.121479


In [85]:
X_test = portfolio_df_lns

In [87]:
columns_to_drop = ['disbursed_month','ln_group']

In [89]:
X_test.columns.tolist()

['disbursed_month',
 'ln_group',
 'disbursed_amount',
 'repaid_amount1',
 'fdr1',
 'bom_proportion',
 'middle_month_proportion',
 'eom_proportion']

In [91]:
## create per ln group
X_test_ln0 = X_test[X_test['ln_group'] == 'ln0']
X_test_ln1 = X_test[X_test['ln_group'] == 'ln1']
X_test_ln2_4 = X_test[X_test['ln_group'] == 'ln2-4']
X_test_ln5_8 = X_test[X_test['ln_group'] == 'ln5-8']
X_test_ln9_12 = X_test[X_test['ln_group'] == 'ln9-12']
X_test_ln13_20 = X_test[X_test['ln_group'] == 'ln13-20']
X_test_ln21 = X_test[X_test['ln_group'] == 'ln21+']
X_test_portfolio = X_test[X_test['ln_group'] == 'Portfolio']

In [93]:
ln_groups = ['ln0', 'ln1', 'ln2_4', 'ln5_8', 'ln9_12', 'ln13_20', 'ln21','portfolio']

In [109]:
# Initialize an empty dictionary to store predictions
predictions = {}

# Loop through the ln groups and make predictions
for ln_group in ln_groups:
    model_key = f'fdr30_forecast_for_dpd1_{ln_group}_model'
    x_test_key = f'X_test_{ln_group}'
    
    # Make predictions for the current ln group
    predictions[ln_group] = model_dict_fdr30_dpd1[model_key].predict(
        globals()[x_test_key].drop(columns=columns_to_drop)
    )

In [111]:
res=pd.DataFrame(predictions).transpose().rename(columns={0:'predicted_fdr30'})
# portfolio_df_lns.set_index('ln_group').join(res)
res

,predicted_fdr30
ln0,21.855165
ln1,11.030482
ln2_4,7.405701
ln5_8,4.491064
ln9_12,4.903209
ln13_20,4.937120
ln21,1.919176
portfolio,5.286777


In [113]:
ln_group_order = ['ln0', 'ln1', 'ln2-4', 'ln5-8', 'ln9-12', 'ln13-20', 'ln21+', 'Portfolio']

In [115]:
portfolio_df_lns['ln_group'] = pd.Categorical(portfolio_df_lns['ln_group'], categories=ln_group_order, ordered=True)


In [117]:
portfolio_df_lns=portfolio_df_lns.sort_values(by='ln_group')
portfolio_df_lns['predicted_fdr30']=res.predicted_fdr30.values
portfolio_df_lns

,disbursed_month,ln_group,disbursed_amount,repaid_amount1,fdr1,bom_proportion,middle_month_proportion,eom_proportion,predicted_fdr30
0,2024-10,ln0,4833977.0,3389862.0,29.874000,18.325386,65.692458,15.982157,21.855165
1,2024-10,ln1,4566943.0,3610004.0,20.954000,17.375246,62.742633,19.882122,11.030482
3,2024-10,ln2-4,13919668.0,11827125.0,15.033000,20.204989,58.212410,21.582602,7.405701
5,2024-10,ln5-8,20289931.0,18130850.0,10.641000,42.837838,17.994088,39.168074,4.491064
6,2024-10,ln9-12,25628793.0,23194776.0,9.497000,46.620116,12.430050,40.949834,4.903209
2,2024-10,ln13-20,50883680.0,46123912.0,9.354000,47.318425,12.619566,40.062009,4.937120
4,2024-10,ln21+,71743360.0,67776442.0,5.529000,44.092488,12.972730,42.934783,1.919176
0,2024-10,Portfolio,191866352.0,174052970.0,9.284266,35.754176,31.124344,33.121479,5.286777


In [119]:
portfolio_df_lns.disbursed_month=portfolio_df_lns.disbursed_month.astype('str')

In [121]:
import datetime

In [123]:
portfolio_df_lns['pred_date']=datetime.date.today().strftime('%Y-%m-%d')

In [125]:
portfolio_df_lns=portfolio_df_lns.drop(columns=['bom_proportion','middle_month_proportion','eom_proportion'])

In [47]:
# # # FIRST UPLOAD ONLY
# portfolio_df_lns.to_sql(table_name, connection, schema='risk_reports',if_exists='replace', index=False)


# Upload to SF

In [48]:
my_user="AMIT_S"
my_pass="Amit2525!!"
my_account="xukvgcv-xm56910"
my_warehouse='COMPUTE_WH'
my_db='ghana_prod'
table_name='fdr_new_predictions'

In [49]:
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

In [50]:
engine = create_engine(URL(user=my_user,password=my_pass,account=my_account,warehouse=my_warehouse,database=my_db))
connection = engine.connect()

In [51]:
# # # FIRST UPLOAD ONLY
portfolio_df_lns.to_sql(table_name, connection, schema='risk_reports',if_exists='append', index=False)


8

# Upload To S3

In [127]:
import boto3

In [129]:
session = boto3.Session(profile_name='gh-dev')

In [131]:
s3 = session.resource('s3')
my_bucket=s3.Bucket('gh-dev-risk-data')

In [ ]:
# creates empty folder

In [190]:
# s3_client = session.client('s3')

# # Specify the bucket name
# bucket_name = 'gh-dev-data-tasks'

# # Specify the folder name you want to create (e.g., 'new-folder/')
# folder_name = 'amit-fdr-predictions/fdr30/'

# # Create the folder by uploading an empty object
# s3_client.put_object(Bucket=bucket_name, Key=folder_name)


In [133]:
# upload df as csv

from io import StringIO

# Assume df is your DataFrame
csv_buffer = StringIO()
portfolio_df_lns.to_csv(csv_buffer, index=False)  # Convert DataFrame to CSV format in-memory


In [135]:
file_name='fdr30_preds_dec2024.csv'

In [137]:

# Create an S3 client
s3_client = session.client('s3')

# Specify the bucket and folder names
bucket_name = 'gh-dev-risk-data'
folder_name = 'fdr-predictions/fdr30/'  # The folder you created in S3

# Upload the CSV to S3
s3_client.put_object(
    Bucket=bucket_name,
    Key=f'{folder_name}{file_name}',  # Path in S3 including the folder
    Body=csv_buffer.getvalue()  # Get the CSV data from the buffer
)


{'ResponseMetadata': {'RequestId': 'MKHETPQ6991D4X27',
  'HostId': 'tp2gRCyWu/NGZcTU3kbLTl38LZGpestD6eyi6LZ82NwkB+W4vQEtrdvRjKCl3HnksHxPAgxJJhT11U7QxKucQbmS7g254nSy',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'tp2gRCyWu/NGZcTU3kbLTl38LZGpestD6eyi6LZ82NwkB+W4vQEtrdvRjKCl3HnksHxPAgxJJhT11U7QxKucQbmS7g254nSy',
   'x-amz-request-id': 'MKHETPQ6991D4X27',
   'date': 'Mon, 06 Jan 2025 09:44:31 GMT',
   'x-amz-version-id': 'hcGUyukS.Fw9gc9ekGZLzqJLvNob._MY',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"35300b432de282580f2a238b4fc97b04"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"35300b432de282580f2a238b4fc97b04"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'hcGUyukS.Fw9gc9ekGZLzqJLvNob._MY'}

# See list of files

In [140]:
!/usr/local/bin/aws --profile gh-dev s3 ls s3://'gh-dev-risk-data/fdr-predictions/fdr30/'


2024-08-28 18:00:47          0 
2024-08-28 18:00:47        670 fdr30_preds_aug2024.csv
2025-01-06 11:44:31        671 fdr30_preds_dec2024.csv
2024-11-03 15:52:22        675 fdr30_preds_oct2024.csv
2024-09-05 16:21:44        668 fdr30_preds_sept2024.csv
